### В этой тетрадке мы объеденили нашу идею, для более понятного её представления.

#### Идея состоит в том, чтобы для работодателей рекомендовать пользователей гитхаба с примерной ценой их работы.

### Загружаем модель

Для решения задачи мы сначала сжали пространство признаков при помощи PCA с 6 компонентами, а потом применили SVR в с rbf kernel над преобразованными данными. Количество компонент и классификатор выбирались по кроссвалидации (см. [FitModel.ipynb](https://github.com/Quest753/ProgrammerRecommendSystem/blob/master/FitModel.ipynb))


<img src="http://what-when-how.com/wp-content/uploads/2011/07/tmp691429_thumb.jpg" alt="Drawing" style="width: 640px; height: 480px"/>


In [59]:
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.decomposition import pca

## Для обучения использовались данные с headhunter'a(см [workWithHH.ipynb](https://github.com/Quest753/ProgrammerRecommendSystem/blob/master/headhunterData/workWithHH.ipynb)) и обучались в [FitModel](https://github.com/Quest753/ProgrammerRecommendSystem/blob/master/FitModel.ipynb)

In [60]:
with open('data.pickle', 'rb') as f:
    svr = pickle.load(f)
with open('pca.pickle','rb') as f:
    pca = pickle.load(f)


In [61]:
def predict(data):
    '''
        Функция предсказывающая цену для каждого пользователя 
            на основе важности требуемых языков программирования.
        data - входные данные для предсказания цены.
        Возвращает предсказанную цену.
    '''
    #уменьшение размерности входных данных
    data_pca = pca.transform(data)
    
    #предсказание логарифма зарплаты при помощи RBF SVM
    prediction = svr.predict(data_pca)

    #пересчитываем логарифм зарплаты обратно в зарплату 
    salary = np.exp(prediction)
    
    return salary



### Считаем тестовые данные
Для того, чтобы предсказываемая цена зависила только от тех языков программирования, которые требуются работодателю, занулим все остальные языки. Тестовые данные берутся из гитхаба, процесс сбора и анализа полученной информации показан в файлах:
* сбор :  [git.ipynb](https://github.com/Quest753/ProgrammerRecommendSystem/blob/master/gitHubData/downloadGitData.ipynb) 
* анализ : [getDataFromGit.ipynb](https://github.com/Quest753/ProgrammerRecommendSystem/blob/master/gitHubData/getDataFromGit.ipynb)

In [90]:
##читаем данные - предобработанную информацию о людях с гита для предсказания цены их работы.
new_data = pd.read_csv("./github/newGitHubUsers.csv")
#передваемые параметры с сайта(Параметры, которые указывает работодатель)
parameters = ["C++","C#",90000]
#запонляем всё нулями because reasons
for col in new_data.columns:
    if col not in parameters:
        new_data[col] = np.zeros(len(new_data))

In [91]:
#Предсказание модели
salary = predict(new_data)

### Составляем таблицу для определения подходящих по цене программистов
Для этого соенияем данные о пользователе с гитхаба( Количество фолловеров, количество просмотров, ссылку на гитхаб) и предсказанную цену.
Затем сортируем выводим пользователей по количеству фолловеров и просмотров. 

In [92]:
#полная информация о пользователях с гита
ranging = pd.read_csv('github/rangingGitLink.csv')
ranging['Salary'] = salary
string = parameters
##выбор пользователей, подходящих под запрос
ans = ranging[(ranging.Salary <= string[-1] * 1.1) & (ranging.Salary >= string[-1] * 0.9)]
ans = ans.sort_values(['Watchers', 'Salary'])[::-1]
#ans.to_csv('answer11111-.csv')

In [93]:
ans[:10]

,Followers,Watchers,Link,Salary
1430,88,7207,https://github.com/ghosthamlet,93375.344154
8912,44,7059,https://github.com/Jackneill,88570.445319
1857,122,4956,https://github.com/pyrotechnick,96033.628983
792,112,4609,https://github.com/rmetzler,98805.241096
1159,102,4322,https://github.com/arden,95529.584930
3879,65,4046,https://github.com/netconstructor,97552.844326
5662,37,3485,https://github.com/yourchanges,84647.897968
15882,170,3478,https://github.com/bfontaine,96876.110802
10526,78,2456,https://github.com/netroby,81960.212363
23106,104,2434,https://github.com/ziyasal,93549.440494


In [81]:
##запрос ["CSS","HTML","JavaScript",80000]
ans[:10]

,Followers,Watchers,Link,Salary
1964,119,6033,https://github.com/frozzare,77976.487697
23571,177,4772,https://github.com/h2non,81522.707178
2800,773,4117,https://github.com/cheeaun,84160.028012
3879,65,4046,https://github.com/netconstructor,77996.219573
8214,281,3677,https://github.com/simplyianm,77011.630974
8616,72,3178,https://github.com/jalbertbowden,83014.328881
3990,747,3146,https://github.com/MatthewMueller,81434.006860
3088,35,2766,https://github.com/heracek,72545.749162
753,39,2691,https://github.com/fabien,82590.961340
5934,61,2388,https://github.com/macressler,83409.110429


In [89]:
# ["SQL", "Go", "Java", 90000]
# коммент что это такое
ans.head()

,Followers,Watchers,Link,Salary
1430,88,7207,https://github.com/ghosthamlet,85261.996543
5758,134,6270,https://github.com/nxbdi,89894.603273
4353,42,5663,https://github.com/wenzhihong2003,83336.515197
1159,102,4322,https://github.com/arden,85672.631147
3879,65,4046,https://github.com/netconstructor,96611.041208
